## Scrapper Automatization

In [31]:
#import library
import requests
from bs4 import BeautifulSoup
from lxml import etree
import pandas as pd


In [11]:
def get_info(url):
    dict={}
    r_notice= requests.get(url)
    s_notice= BeautifulSoup(r_notice.text, 'lxml')

    #title notice
    title=s_notice.find('h1', attrs={'class':'title'}).text
    if title:
        dict['title']=title
    else:
        dict['title']=None

    #date notice
    time=s_notice.find('time', attrs={'class':'date'}).text
    if time:
        dict['time']=time
    else:
        dict['time']=None

    #lead
    lead=s_notice.find('h2', attrs={'class':'excerpt'}).text
    if time:
        dict['lead']=lead
    else:
        dict['lead']=None
    
    #body
    b=''
    body=s_notice.find('div', attrs={'class':'body-content note-body'}).find_all('p')
    for p in body:
        b=b+''+(p.text)
    if b:
        dict['body']=b
    else:
        dict['body']=None

    #img
    media = s_notice.find('div', attrs={'class':'image'}).find('img')
    img_src=media.get('data-td-src-property')
    try:
        r_img=requests.get(img_src)
        if r_img.status_code==200:
            dict['img']=r_img.content
        else:
            dict['img']=None
    except:
        print('there is not image')

    return dict




In [12]:
get_info('https://www.ultimahora.com/corte-rechaza-accion-contra-ordenanza-que-limita-distancia-gasolineras-asuncion-n3040554.html')

{'title': 'Corte rechaza acción contra ordenanza que limita distancia de gasolineras en Asunción',
 'time': '26 de diciembre de 2022 - 10:10',
 'lead': 'La Corte Suprema de Justicia rechazó una acción de inconstitucionalidad contra la ordenanza municipal que establece limitaciones de distancia para las estaciones de servicio en Asunción. Con esto, se deja sin efecto la medida cautelar que habían obtenido algunas gasolineras.',
 'body': 'El recurso fue planteado por el abogado Juan Emmanuel Martínez Sosa, en representación de la firma Cámara de Distribuidoras Paraguayas de Combustibles (Cadipac).Con esto, los ministros Antonio Fretes, Víctor Ríos y César Diésel resolvieron ordenar el levantamiento de la medida de suspensión de efectos de la ordenanza municipal, que se había resuelto en febrero de 2021 a favor del gremio, según confirmó el director de Gabinete de la Comuna, Federico Mora.No obstante, el funcionario señaló que son cinco las firmas que presentaron el mismo pedido de amparo

In [14]:
def scraper_url(url):
    try:
        r=requests.get(url)
    except Exception as e:
        print('Error ',url)
        print (e)
        return None
    
    if r.status_code !=200:
        print(f'Error status code {url}')
        print(e)
        return None
    
    dict=get_info(url)
    dict['url']=url

    return dict

In [15]:
scraper_url('https://www.ultimahora.com/corte-rechaza-accion-contra-ordenanza-que-limita-distancia-gasolineras-asuncion-n3040554.html')

{'title': 'Corte rechaza acción contra ordenanza que limita distancia de gasolineras en Asunción',
 'time': '26 de diciembre de 2022 - 10:10',
 'lead': 'La Corte Suprema de Justicia rechazó una acción de inconstitucionalidad contra la ordenanza municipal que establece limitaciones de distancia para las estaciones de servicio en Asunción. Con esto, se deja sin efecto la medida cautelar que habían obtenido algunas gasolineras.',
 'body': 'El recurso fue planteado por el abogado Juan Emmanuel Martínez Sosa, en representación de la firma Cámara de Distribuidoras Paraguayas de Combustibles (Cadipac).Con esto, los ministros Antonio Fretes, Víctor Ríos y César Diésel resolvieron ordenar el levantamiento de la medida de suspensión de efectos de la ordenanza municipal, que se había resuelto en febrero de 2021 a favor del gremio, según confirmó el director de Gabinete de la Comuna, Federico Mora.No obstante, el funcionario señaló que son cinco las firmas que presentaron el mismo pedido de amparo

In [16]:
def get_next_page_links(url):
    list=[url]

    for i in range(5):
        r= requests.get(list[-1])
        s= BeautifulSoup(r.text, 'lxml')
        get_next_page= s.find('a', attrs={'class':'right-button'})
        get_link=get_next_page.get('href')
        list.append(get_link)
    
    return list

In [17]:
get_next_page_links('https://www.ultimahora.com/contenidos/nacional.html')

['https://www.ultimahora.com/contenidos/nacional.html',
 'https://www.ultimahora.com/contenidos/nacional.html/1',
 'https://www.ultimahora.com/contenidos/nacional.html/2',
 'https://www.ultimahora.com/contenidos/nacional.html/3',
 'https://www.ultimahora.com/contenidos/nacional.html/4',
 'https://www.ultimahora.com/contenidos/nacional.html/5']

In [25]:
def get_notice_links(url):
    all_link=[]
    for i in url:
        try:
            r=requests.get(i)
            s=BeautifulSoup(r.text, 'lxml')
            xp_url_section= etree.HTML(str(s))
            links_notices=xp_url_section.xpath("//div[@class='col-12 col-lg-8']//*[@class='article-title']//a")
            for i in links_notices:
                link=i.get("href")
                all_link.append(link)
                
        except Exception as e:
            print ('Error \n')
            print(e)
    
    return all_link

In [26]:
get_notice_links(get_next_page_links('https://www.ultimahora.com/contenidos/nacional.html'))

['https://www.ultimahora.com/hospital-trauma-atiende-mas-1700-pacientes-nochebuena-y-navidad-n3040560.html',
 'https://www.ultimahora.com/musico-denuncia-agresion-policial-aprehension-irregular-y-extorsion-luque-n3040550.html',
 'https://www.ultimahora.com/lago-ypacarai-es-como-banarse-un-inodoro-advierte-directora-del-mades-n3040552.html',
 'https://www.ultimahora.com/patrulla-caminera-demoro-155-conductores-alcoholizados-navidad-n3040542.html',
 'https://www.ultimahora.com/bebe-prematuro-necesita-urgencia-una-cama-terapia-intensiva-n3040544.html',
 'https://www.ultimahora.com/aparatoso-vuelco-vehiculo-deja-un-fallecido-y-cuatro-heridos-concepcion-n3040543.html',
 'https://www.ultimahora.com/clima-caluroso-chaparrones-y-tormentas-continuaran-este-lunes-n3040540.html',
 'https://www.ultimahora.com/lo-que-tenes-que-saber-comenzar-tu-lunes-n3040460.html',
 'https://www.ultimahora.com/accidentes-dejan-tres-motociclistas-muertos-esta-navidad-n3040449.html',
 'https://www.ultimahora.com/rep

In [ ]:
list_notice='https://www.ultimahora.com/contenidos/nacional.html'
for i in range(10):
    r= requests.get(list_notice[-1])
    s=BeautifulSoup(r.text, 'lxml')
    get_next_page= s.find('a', attrs={'class':'right-button'})
    get_link=get_next_page.get('href')



In [27]:
#scrapper
data=[]
all_links=get_notice_links(get_next_page_links('https://www.ultimahora.com/contenidos/nacional.html'))
for contador, link in enumerate(all_links):
    print (f'scraper link: {contador}/{len(all_links)}')
    data.append(scraper_url(link))

scraper link: 0/66
scraper link: 1/66
scraper link: 2/66
scraper link: 3/66
scraper link: 4/66
scraper link: 5/66
scraper link: 6/66
scraper link: 7/66
scraper link: 8/66
scraper link: 9/66
scraper link: 10/66
scraper link: 11/66
scraper link: 12/66
scraper link: 13/66
scraper link: 14/66
scraper link: 15/66
scraper link: 16/66
scraper link: 17/66
scraper link: 18/66
scraper link: 19/66
scraper link: 20/66
scraper link: 21/66
scraper link: 22/66
scraper link: 23/66
scraper link: 24/66
scraper link: 25/66
scraper link: 26/66
scraper link: 27/66
scraper link: 28/66
scraper link: 29/66
scraper link: 30/66
scraper link: 31/66
scraper link: 32/66
scraper link: 33/66
scraper link: 34/66
scraper link: 35/66
scraper link: 36/66
scraper link: 37/66
scraper link: 38/66
scraper link: 39/66
scraper link: 40/66
scraper link: 41/66
scraper link: 42/66
scraper link: 43/66
scraper link: 44/66
scraper link: 45/66
scraper link: 46/66
scraper link: 47/66
scraper link: 48/66
scraper link: 49/66
scraper li

In [28]:
len(data)

66

In [33]:
df = pd.DataFrame(data)
df

,title,time,lead,body,img,url
0,Se realiza la primera reconstrucción pélvica y...,26 de diciembre de 2022 - 12:45,Se trata de la realización de una nueva vejiga...,Los cirujanos oncólogos Emilio Brizuela y Seba...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,https://www.ultimahora.com/se-realiza-la-prime...
1,Hospital de Trauma atiende a más de 1.700 paci...,26 de diciembre de 2022 - 10:27,El Hospital de Trauma cerró el fin de semana c...,En esos dos días predominaron los casos de acc...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,https://www.ultimahora.com/hospital-trauma-ati...
2,Corte rechaza acción contra ordenanza que limi...,26 de diciembre de 2022 - 10:10,La Corte Suprema de Justicia rechazó una acció...,El recurso fue planteado por el abogado Juan E...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,https://www.ultimahora.com/corte-rechaza-accio...
3,Armada minimiza situación del cañonero Humaitá...,26 de diciembre de 2022 - 09:23,La Armada Paraguaya negó que el histórico caño...,"Sin embargo, el contralmirante Carlos Díaz, co...",b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,https://www.ultimahora.com/armada-minimiza-sit...
4,"Músico denuncia agresión policial, aprehensión...",26 de diciembre de 2022 - 09:12,"Un bajista local, que toca música jazz, denunc...",En las redes sociales comentó que se encontrab...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,https://www.ultimahora.com/musico-denuncia-agr...
...,...,...,...,...,...,...
61,Intenso movimiento en paso de Encarnación y Po...,24 de diciembre de 2022,Unas 50.000 personas cruzaron el paso entre En...,El gran movimiento se intensificó desde el últ...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,https://www.ultimahora.com/intenso-movimiento-...
62,Motochorros balean a hombre que se resistió a ...,24 de diciembre de 2022,Cuatro motochorros intentaron asaltar a un hom...,"Según informes, el hecho ocurrió aproximadamen...",b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,https://www.ultimahora.com/motochorros-balean-...
63,Movimiento en Estación de Buses no cesa y supe...,24 de diciembre de 2022,El movimiento de pasajeros en la Estación de B...,"“Cerramos una jornada con 30.500 personas, que...",b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,https://www.ultimahora.com/movimiento-estacion...
64,Hombre de 58 años fallece en accidente de trán...,24 de diciembre de 2022,Un accidente con derivación fatal ocurrió en l...,La víctima fatal fue identificada como Silveri...,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00...,https://www.ultimahora.com/hombre-58-anos-fall...


In [35]:
df.to_excel('scraper.xlsx')